In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image
import imutils
from skimage.filters import threshold_otsu
import skimage.io as io
import math
import csv
from matplotlib.patches import Rectangle
import os



HOST = "192.168.125.5" #my IP address
SERVER_PORT = 65432
FORMAT = "utf8"

In [34]:
def neighbours(x,y,image): #for thining algorithm
    "Return 8-neighbours of image point P1(x,y), in a clockwise order"
    img = image
    x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
    return [ img[x_1][y], img[x_1][y1], img[x][y1], img[x1][y1],     # P2,P3,P4,P5
                img[x1][y], img[x1][y_1], img[x][y_1], img[x_1][y_1] ]    # P6,P7,P8,P9

def transitions(neighbours): #for thining algorithm
    "No. of 0,1 patterns (transitions from 0 to 1) in the ordered sequence"
    n = neighbours + neighbours[0:1]      # P2, P3, ... , P8, P9, P2
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )  # (P2,P3), (P3,P4), ... , (P8,P9), (P9,P2)

def zhangSuen(image): #main of thining algorithm
    "the Zhang-Suen Thinning Algorithm"
    Image_Thinned = image.copy()  # deepcopy to protect the original image
    changing1 = changing2 = 1        #  the points to be removed (set as 0)
    while changing1 or changing2:   #  iterates until no further changes occur in the image
        # Step 1
        changing1 = []
        rows, columns = Image_Thinned.shape               # x for rows, y for columns
        for x in range(1, rows - 1):                     # No. of  rows
            for y in range(1, columns - 1):            # No. of columns
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1     and    # Condition 0: Point P1 in the object regions 
                    2 <= sum(n) <= 6   and    # Condition 1: 2<= N(P1) <= 6
                    transitions(n) == 1 and    # Condition 2: S(P1)=1  
                    P2 * P4 * P6 == 0  and    # Condition 3   
                    P4 * P6 * P8 == 0):         # Condition 4
                    changing1.append((x,y))
                    
        for x, y in changing1: 
            Image_Thinned[x][y] = 0
            
        #Step 2
        changing2 = []
        for x in range(1, rows - 1):
            for y in range(1, columns - 1):
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1   and        # Condition 0
                    2 <= sum(n) <= 6  and       # Condition 1
                    transitions(n) == 1 and      # Condition 2
                    P2 * P4 * P8 == 0 and       # Condition 3
                    P2 * P6 * P8 == 0):            # Condition 4
                    changing2.append((x,y))    
        for x, y in changing2: 
            Image_Thinned[x][y] = 0
    return Image_Thinned

def find_start_point(final): #find start point in roaming_path
    rows, cols = final.shape
    for i in range(cols):
        if sum(final[:, i]) != 0:
            for y in range(len(final[:, i])):
                if final[y,i] == 1 and sum(sum(final[y-1:y+2,i-1:i+2])) <= 2:
                    return y,i

def roaming_path(matrix, start_point): #roaming path to extract ordered path - R_arm
    matrix_ = matrix.copy() #the original matrix 
    path = [start_point] #init an array that contain 1st ele is start_point
    while sum(sum(matrix_)) !=0: # to loop untill no vacant position
        
        check_expand = 0

        x = path[-1] # to move cursor to end of array
        
        breaker0 = False
        breaker01 = False
        breaker02 = False
        a = 0 
        for a in range(1,340): #expand the search window to find the next window
            check_expand += 1
            #adjust 4 corners of window
            m00 = x[0]-a
            if m00 < 0:
                m00 = 0
            m01 = x[0]+a
            if m01 >= matrix_.shape[0]:
                m01 = matrix_.shape[0]
            m10 = x[1]-a
            if m10 < 0:
                m10 = 0
            m11 = x[1]+a
            if m11 >= matrix_.shape[1]:
                m11 = matrix_.shape[1]

            #if array is a single, will break the expand window
            if sum(sum(matrix_[m00:m01+ 1, m10: m11+ 1])) >= 1 and a == 1:
                break
            
            #if arry is a ele left, break
            if sum(sum(matrix_)) == 1 or sum(sum(matrix_)) == 0:
                break


            #if total of window value is high than 1, will implement below code
            elif sum(sum(matrix_[m00:m01+ 1, m10: m11+ 1])) >= 1:
                
                #if do to Good position, will append to array and will break window loop
                for i in range(m00, m01 ):
                    for j in range(m10, m11 ):

                        if matrix_[i,j] != 0 and i != x[0] and j != x[1]:
                            path.append([i,j])   
                            matrix_[i,j] = 0
                            if sum(sum(matrix_[i-1:i+2,j-1:j+2])) >1:
                                breaker0 = True
                                breaker01 = True
                                breaker02 = True
                            break
                    if breaker0:
                        break
                if breaker02:
                        break
            if breaker01:
                break

        distance_init = 10000
        x = path[-1]
        breaker1 = False
        breaker2 = False
        y = [None, None]
        
        for i in range(x[0] -1, x[0] +1 +1):
            
            for j in range(x[1] -1, x[1] +1 +1):
                #do not implement to center window point
                if i == x[0] and j == x[1]:
                    continue

                if matrix_[i,j] != 0:
                    
                    #if is first point that is start_point
                    if len(path)==1:
                        y = [i,j]
                        breaker1 = True
                        break

                    else:
                        #tìm điểm gần nhất 
                        d = math.sqrt(pow((x[0]-i), 2) + pow((x[1]-j), 2))
                        if d <= distance_init:
                            y = [i, j]
                            distance_init = d
            if breaker1:
                break
        matrix_[y[0], y[1]] = 0
        
        if y != [None, None]:
            path.append(y)
        #print(len(path), sum(sum(matrix_)), check_expand)
        
    return path
               
def min_thin_ver2(point_pos, mask_test, window_size = 21): # to extract size of pen point
    test_matrix = mask_test[int(point_pos[0]- (window_size-1)/2):int(point_pos[0] + (window_size-1)/2 + 1),
                            int(point_pos[1] - (window_size-1)/2):int(point_pos[1] + (window_size-1)/2 + 1)]
    #test_matrix = test_matrix.astype(int)
    sum_indices = window_size

    mask_1 = np.zeros([window_size,window_size], dtype="uint8")
    mask_1[:,int(window_size/2)] = 1

    mask_2 = np.zeros([window_size,window_size], dtype="uint8")
    mask_2[int(window_size/2),:] = 1
    # diagonal matrix
    mask_3 = np.eye(window_size,window_size)
    mask_3 = mask_3.astype(int)
    # sub diagonal matrix
    mask_4 = np.zeros([window_size,window_size], dtype="uint8")
    i,j = np.indices(mask_4.shape)
    mask_4[i == window_size - 1 - j] = 1


    #window_size = 21
    mask_5 = np.zeros([window_size,window_size], dtype="uint8")
    i,j = np.indices(mask_5.shape)
    mask_5[i-np.floor((window_size-2)/2) == window_size - j*2] = 1
    mask_5[1,14]=1
    mask_5[3,13]=1
    mask_5[5,12]=1
    mask_5[7,11]=1
    mask_5[9,10]=1
    mask_5[10,10]=1
    mask_5[11,10]=1
    mask_5[13,9]=1
    mask_5[15,8]=1
    mask_5[17,7]=1
    mask_5[19,6]=1

    #window_size = 21
    mask_6 = np.zeros([window_size,window_size], dtype="uint8")
    i,j = np.indices(mask_6.shape)
    #mask_6[i-np.floor((window_size-2)/2) == window_size - j*2] = 1
    mask_6[j-np.floor((window_size-2)/2) == window_size - i*2] = 1
    mask_6[5,19]=1
    mask_6[6,17]=1
    mask_6[7,15]=1
    mask_6[8,13]=1
    mask_6[9,11]=1
    mask_6[10,10]=1
    mask_6[11,9]=1
    mask_6[12,7]=1
    mask_6[13,5]=1
    mask_6[14,3]=1
    mask_6[15,1]=1

    mask_7 = np.flip(mask_6,0)

    mask_8 = np.flip(mask_5,0)

    mask_list = [mask_1, mask_2, mask_3, mask_4, mask_5, mask_6, mask_7, mask_8]
    for i in mask_list:
        tmp = sum(sum(test_matrix * i))
        if tmp < sum_indices:
            sum_indices = tmp
    return sum_indices

def create_coordination_no_visual(text_input, row = 100, column = -40): # create path for R-arm procedure
    mask = np.zeros([155,340,3], dtype="uint8")
    
    fontpath = "/Users/anhduc/Documents/Project_Program/Final_Project/font/UTM ONG DO GIA.TTF"
    font = ImageFont.truetype(fontpath,140)
    img_pil = Image.fromarray(mask)
    draw = ImageDraw.Draw(img_pil)
    draw.text((row, column),  text_input, font = font, fill = (0, 255, 0))

    img = np.array(img_pil)
    img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    
    mask_ = np.full((155,340), False)
    mask_[img != 0] = True

    final = zhangSuen(mask_)

    arr = roaming_path(final, find_start_point(final))
    mask_test = mask_.copy()

    #print(min_thin(arr[0]))
    arr_thining = []
    for j, i in enumerate(arr):
        if j%1==0:
            arr_thining.append([i[0],i[1],min_thin_ver2(i,mask_test)])
    return arr_thining

def add_split(arr_thining): # add point[*,*,0] to split uncontinue line
    arr_thining_ = []
    #append first point with z = 0
    arr_thining_.append([arr_thining[0][0],arr_thining[0][1],0])

    for num in range(0,len(arr_thining)-1):
        d = abs(np.sqrt(pow((arr_thining[num+1][0]-arr_thining[num][0]),2)+pow((arr_thining[num+1][1]-arr_thining[num][1]),2)))
        arr_thining_.append([arr_thining[num][0],arr_thining[num][1],arr_thining[num][2]])
        if d > 5:
            #print(num)
            arr_thining_.append([arr_thining[num][0],arr_thining[num][1],0])
            arr_thining_.append([arr_thining[num+1][0],arr_thining[num+1][1],0])
    arr_thining_.append([arr_thining[-1][0],arr_thining[-1][1],arr_thining[-1][2]])
    
    #append last point with z = 0
    arr_thining_.append([arr_thining[-1][0],arr_thining[-1][1],0])
    return arr_thining_

def calib(arr_thining_):
    arr_thining_1 = arr_thining_.copy()
    arr_thining_2 = np.array(arr_thining_1)

    # h_origin = 300 #mm  278.17; -168.86  --> 356.27; -58.96
    # w_origin = 533

    h_origin = 103*1.8
    w_origin = 222*1.8

    h_pixel = 155 # image  162,288
    w_pixel = 340


    arr_thining_2[:,0] = (np.floor((arr_thining_2[:,0]/h_pixel)*h_origin)).astype(int)
    arr_thining_2[:,1] = (np.floor((arr_thining_2[:,1]/w_pixel)*w_origin)).astype(int)

    arr_Z = []

    for i in range(len(arr_thining_2[:,2])):
        if arr_thining_2[:,2][i] == 0:
            arr_Z.append(1)
        else:
            arr_Z.append(int((arr_thining_2[:,2][i]-1)*(-11/19)))
            
    arr_thining_3 = np.stack((arr_thining_2[:,0],arr_thining_2[:,1],arr_Z), axis = 1)

    return arr_thining_3

def cvt_2_string(arr_int):
    arr_str = []
    for i in range( 0, len(arr_int)):
        arr_str.append(str(arr_int[i][0])+";"+str(arr_int[i][1])+";"+str(arr_int[i][2]))
    return arr_str

def cvt_2_string_2_arm(arr_int):
    arr_str = []
    for i in range( 0, len(arr_int)):
        arr_str.append(str(arr_int[i][0])+";"+str(arr_int[i][1])+";"+str(arr_int[i][2])+";"+str(arr_int[i][3])+";"+str(arr_int[i][4])+";"+str(arr_int[i][5]))
    return arr_str   

def final_procedure_sign(caligraphy_text):

    for i in range(len(caligraphy_text)):

        if i == 0:
            arr_1 = create_coordination_no_visual(caligraphy_text[i], row = 25, column = -50)
            arr_1_split = add_split(arr_1)
            arr_1_calib = calib(arr_1_split)
            arr_1_str = cvt_2_string(arr_1_calib)
            #hienthi2(arr_1)
        elif i == 1:
            arr_2 = create_coordination_no_visual(caligraphy_text[i], row = 90, column = -50)
            arr_2_split = add_split(arr_2)
            arr_2_calib = calib(arr_2_split)
            arr_2_str = cvt_2_string(arr_2_calib)
            #hienthi2(arr_2)
        elif i == 2:
            arr_3 = create_coordination_no_visual(caligraphy_text[i], row = 40+45*i, column = -50)
            arr_3_split = add_split(arr_3)
            arr_3_calib = calib(arr_3_split)
            arr_3_str = cvt_2_string(arr_3_calib)
            #hienthi2(arr_3)
        elif i == 3:
            arr_4 = create_coordination_no_visual(caligraphy_text[i], row = 40+45*i, column = -50)
            arr_4_split = add_split(arr_4)
            arr_4_calib = calib(arr_4_split)
            arr_4_str = cvt_2_string(arr_4_calib)
            #hienthi2(arr_4)
        else:
            arr_5 = create_coordination_no_visual(caligraphy_text[i], row = 40+45*i, column = -50)
            arr_5_split = add_split(arr_5)
            arr_5_calib = calib(arr_5_split)
            arr_5_str = cvt_2_string(arr_5_calib)
            #hienthi2(arr_5)
    #hienthi2(arr_1+ arr_2+ arr_3+ arr_4+ arr_5)
    new = arr_1
    print(len(new))
    if len(caligraphy_text) == 1:
        return arr_1_str,["Done"],["000000"]
    elif len(caligraphy_text) == 2:
        return arr_1_str,["End_C"], arr_2_str,["Done"],["000000"]
    elif len(caligraphy_text) == 3:
        return arr_1_str, ["End_C"],arr_2_str,arr_3_str,["Done"],["000000"]
    elif len(caligraphy_text) == 4:
        return arr_1_str, ["End_C"],arr_2_str,arr_3_str, ["End_C"],arr_4_str,["Done"],["000000"]
    else:
        return arr_1_str, ["End_C"],arr_2_str,arr_3_str, ["End_C"],arr_4_str,arr_5_str,["Done"],["000000"]

def hienthi2(trajectory):
    print(trajectory)
    arr = np.array(trajectory)

    # Tính toán phạm vi của dữ liệu
    x_min, x_max = np.min(arr[:, 0]), np.max(arr[:, 0])
    y_min, y_max = np.min(arr[:, 1]), np.max(arr[:, 1])

    # Tính toán kích thước hình vẽ dựa trên tỷ lệ giữa phạm vi x và y và một hệ số (vd: 0.1)
    x_range = x_max - x_min
    y_range = y_max - y_min
    aspect_ratio = y_range / x_range
    figsize = (7 * aspect_ratio, 7)

    plt.figure(figsize=figsize,facecolor='black')
    cl = 0
    cl_ = 0
    colors = np.array(["red", "green", "blue", "yellow", "orange", "purple", "gray", "magenta", "pink"])
    col_step_ = np.ceil(len(arr) / len(colors))
    col_step = col_step_
    for i in arr:
        if i[0] != 21:
            if cl > col_step:
                col_step += col_step_
                cl_ += 1
            plt.scatter([i[1]], [i[0]], color=colors[cl_], linewidths=0.1)
            cl += 1
    plt.gca().invert_yaxis()
    plt.axis('off')
    plt.show()


In [35]:

trajectory = final_procedure_sign("a")


127


In [ ]:
#----------------------main---------------------
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print("CLIENT SIDE")

try:
    client.connect( (HOST, SERVER_PORT) )
    print("client address:",client.getsockname())

    start_sig = ' '
    flag_routine = 0 
    
    while (flag_routine==0 and start_sig != "x"):

        flag_process_img = None 
        start_sig = input("Do you want to start the routine: \nPlease type 'Start' or 'x' or 'r_arm'")
        center_shape_ = []
        center_shape_sign = []

        if start_sig == "Start": #Nếu nhận "Start"
            flag_process_img_ = "Đức"
            center_shape_sign = final_procedure_sign(flag_process_img_)
            flag_routine = 1
            client.sendall(start_sig.encode(FORMAT)) #gui tin hieu "Start"
            print("Client has send the signal 'Start' \n --------------------")

            while flag_process_img != 'x' and flag_process_img != "End":
                print("wait responsible signal...")
                flag_process_img = client.recv(1024).decode(FORMAT) #Doi ABB tra ve "Done_Capture"

                client.sendall('Done_Processing'.encode(FORMAT))
                sig_return = "_"
                for i in center_shape_sign:
                    for j in i:
                        while str(sig_return) != "*" and str(sig_return)  != "**" :
                            sig_return = client.recv(1024).decode(FORMAT)
                        client.sendall(j.encode(FORMAT))
                        # print(j)
                        sig_return = "_"
    
                while str(sig_return) != "*":
                    sig_return = client.recv(1024).decode(FORMAT)
                client.sendall("Done_Write".encode(FORMAT))
                time.sleep(0.5)
            
                print("Done Routine")
                flag_process_img = "End"
                client.sendall("Done_Write".encode(FORMAT))
                flag_routine = 0
               
except:
    print("Error")

client.close();